<div style="text-align: center;"> <h1>GROUP F MACHINE LEARNING FOR DATA SCIENCE FINAL PROJECT</h1><> </div>

This Jupyter Notebook will be the main program from our project. When the project is finished, we will be able to use this Jupyter code to generate the presentation slides.

In [1]:
import Scripts #Import all the scripts in the "Scripts" folder
import pandas as pd
import torch 
import numpy as np
import matplotlib as plt

The first step to make is to import our training data and store it into a pandas structure and start the data preprocessing.

In [2]:
dataPath = "./Data/TRAIN_Vehicles_Data.xlsx" #The route to the data
dataT = pd.read_excel(dataPath, na_values=["N", " "]) #After seeing the excel we set the null values as "N" and " "


Step 1) By inspection, remove all the columns that we stimate that will be not useful.

In [3]:
dataT = dataT.drop(columns=["RPAKREP_VEHICLE_HKEY","CHASSIS_NUMBER","OFFICE_MAIN_BRAND","MANUFACTURER",
                            "VEHICLE_TYPE","VARIANT","RIM_KEY","UPHOLSTERY_CODE","ENGINE_ID","TRANSMISSION_TYPE",
                            "COMMISSION_NUMBER","LEASING_START","LEASING_END","LEASING_MILAGE","FINANCING_TYPE_NAME",
                            "PURCHASE_DATE","PRICE_LIST","SOLD_INVOICE_COSTUMER_ID","SOLD_CUSTOMER_ID"])


Step 2) Eliminate the columns with more 40% of its values being null. 

In [4]:

null_percentage = dataT.isnull().mean() * 100 #Multiply the mean by 100 to pass to percentage
threshold = 40 #40% of null entries
columns_to_drop = null_percentage[null_percentage > threshold].index #Extract the index of the columns with more of 40% of its values being null
dataT = dataT.drop(columns=columns_to_drop) #Drop the columns
print("\nDataFrame after dropping columns with more than 40% null values:")
print(dataT)




DataFrame after dropping columns with more than 40% null values:
       COMPANY OFFICE MANUFACTURER_SHORT VEHICLE_GROUP  MODEL_CODE  MILEAGE  \
0         44.0     TA                FOR           ECS         NaN   8600.0   
1         12.0     44                VOL           XC4  536K7170F1      0.0   
2         27.0     33                FOR           TOC          26   1297.0   
3         33.0     K1                  V           POL      AE13GV   6020.0   
4         10.0     96                FOR           FOC          15      0.0   
...        ...    ...                ...           ...         ...      ...   
99066     70.0      4                JAG           EPA         NaN     20.0   
99067     17.0     60                FOR           FIE          13  67748.0   
99068     33.0     KA                  V           AMA      T1BE76   3549.0   
99069     10.0     10                FOR           TNT          85  14117.0   
99070     17.0     61                ALF           STE         Na

Step 3) Statistic description of the data

In [5]:
pd.set_option('display.max_columns', None) #To see all the columns
dataT.describe()

,COMPANY,MILEAGE,OPERATING_HOURS,MILAGE_IN_FIELD,MILAGE_SALES,OPERATING_HOURS_SALES,CERTIFICATE_TYPE_DATE,FRONT_TIRES_CONDITION,REAR_TIRES_CONDITION,NUMBER_DOORS,NUMBER_SEATS,PERMITTED_TOTAL_WEIGHT,MAX_TRAILOR_LOAD,CURB_WEIGHT,YEAR_CONSTRUCTION,CONSTRUCTION_MONTH,NUMBER_AXLE,NUMBER_ENGINE_CYLINDER,REPAIR_RKZ,OPTICAL_CONDITION,TECHNICAL_CONDITION,HORSEPOWER,KW,CCM,NUMBER_OWNERS,IS_USED_CAR,FINANCING_TYPE,DRIVE_TYPE,VEHICLE_MODEL_ID,PURCHASE_MILAGE,PURCHASE_OPERATION_HOURS,AT_LOCATION_SINCE,LAID_UP_TIME,MILAGE_SALE,OPERATION_HOURS_SALE,SCALED_CURRENT_VALUE,SCALED_INVENTURAL_VALUE,SCALED_REPORT_VALUE,SCALED_VALUATION_PRICE,SCALED_GUIDE_PRICE,SCALED_TOTAL_SALES_PRICE_BASIS,SCALED_TOTAL_SALE_PRICE
count,99040.000000,9.904000e+04,99040.000000,99038.0,9.904000e+04,99040.000000,99004.0,99004.0,99004.0,99004.000000,99004.000000,99004.000000,99004.000000,99004.000000,74366.000000,70320.000000,99004.000000,99004.000000,99004.0,73107.000000,73057.0,99004.000000,99004.000000,99004.000000,99004.000000,99004.000000,96117.000000,64514.000000,99004.000000,9.900400e+04,99004.0,9.900400e+04,99004.000000,9.900400e+04,99004.000000,9.900400e+04,99004.000000,99004.000000,99004.0,9.900400e+04,9.900400e+04,9.900400e+04
mean,22.375808,3.071296e+04,0.275535,1.0,2.439775e+04,0.275374,0.0,0.0,0.0,3.432962,3.439932,243.442265,77.284766,1075.761202,2020.570059,6.360922,0.580775,2.544826,0.0,0.000041,0.0,149.995606,110.888772,1522.064715,0.569836,0.459699,1.720944,1.557615,3.010757,2.210805e+04,0.0,3.651454e+05,88.271939,2.422813e+04,0.275464,1.010119e+03,0.013350,0.944832,0.0,2.020125e+03,1.010080e+03,1.010078e+03
std,13.131775,1.873386e+05,56.329787,0.0,9.623303e+04,56.329776,0.0,0.0,0.0,2.161191,2.359111,662.282475,364.058658,872.056447,26.109127,3.517440,0.907870,2.031409,0.0,0.011095,0.0,107.488948,78.410067,2533.676627,0.721395,0.498376,1.063054,0.906579,2.617623,7.355878e+04,0.0,5.631298e+05,103.060790,1.008774e+05,56.340016,3.178144e+05,0.031157,0.030980,0.0,4.494552e+05,3.178144e+05,3.178144e+05
min,10.000000,0.000000e+00,0.000000,1.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000,-1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,-119.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,2.980100e-04
25%,12.000000,1.000000e+01,0.000000,1.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2020.000000,3.000000,0.000000,0.000000,0.0,0.000000,0.0,109.000000,81.000000,999.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000e+00,0.0,0.000000e+00,22.000000,0.000000e+00,0.000000,5.178134e-02,0.000000,0.950043,0.0,2.375720e-03,1.087726e-02,1.064459e-02
50%,18.000000,5.285000e+03,0.000000,1.0,3.003000e+03,0.000000,0.0,0.0,0.0,5.000000,5.000000,0.000000,0.000000,1345.000000,2022.000000,6.000000,0.000000,3.000000,0.0,0.000000,0.0,145.000000,110.000000,1498.000000,0.000000,0.000000,1.000000,1.000000,3.000000,3.000000e+00,0.0,0.000000e+00,59.000000,1.000000e+02,0.000000,5.704270e-02,0.000000,0.950043,0.0,4.373960e-03,1.805227e-02,1.578809e-02
75%,33.000000,2.694025e+04,0.000000,1.0,2.400000e+04,0.000000,0.0,0.0,0.0,5.000000,5.000000,0.000000,0.000000,1748.000000,2023.000000,10.000000,2.000000,4.000000,0.0,0.000000,0.0,190.000000,137.000000,1995.000000,1.000000,1.000000,3.000000,2.000000,5.000000,2.163450e+04,0.0,1.230314e+06,127.000000,2.377125e+04,0.000000,6.292749e-02,0.000000,0.950043,0.0,6.094278e-03,2.583623e-02,2.188531e-02
max,72.000000,9.960005e+06,17140.000000,1.0,9.999999e+06,17140.000000,0.0,0.0,0.0,5.000000,23.000000,20075.000000,3500.000000,19855.000000,2033.000000,12.000000,2.000000,12.000000,0.0,3.000000,0.0,13067.000000,9611.000000,98700.000000,9.000000,1.000000,5.000000,9.000000,9.000000,9.000009e+06,0.0,1.260104e+06,8782.000000,9.999999e+06,17140.000000,1.000000e+08,0.871569,0.963855,0.0,1.000000e+08,1.000000e+08,1.000000e+08


Step 4) Visualization of the variable type of the attributes

In [6]:
pd.set_option('display.max_rows', None) #To see all the rows
dataT.dtypes

COMPANY                                  float64
OFFICE                                    object
MANUFACTURER_SHORT                        object
VEHICLE_GROUP                             object
MODEL_CODE                                object
MILEAGE                                  float64
OPERATING_HOURS                          float64
MILAGE_IN_FIELD                          float64
MILAGE_SALES                             float64
OPERATING_HOURS_SALES                    float64
COLOR_CODE                                object
COLOR_CODE_NAME                           object
COLOR                                     object
UPHOLSTERY                                object
CERTIFICATE_TYPE_DATE                    float64
ENGINE_TYPE                               object
TRANSMISSION_SHORT                        object
TRANSMISSION_NAME                         object
FRONT_TIRES_CONDITION                    float64
REAR_TIRES_CONDITION                     float64
NUMBER_DOORS        